# Clean and Prepare Sudan Climate Data

## Introduction

To support the ELO2 project’s goal of analyzing climate patterns in Sudan, this notebook focuses on **cleaning and preparing temperature and rainfall datasets** collected from the [NASA POWER Data Access Viewer](https://power.larc.nasa.gov/).

Five regional datasets were selected to represent distinct climate zones:
- **North:** Desert zone (Dongola)
- **Central:** Semi-arid zone (Khartoum, Gezira)
- **East:** Coastal and semi-arid zone (Kassala, Port Sudan)
- **West:** Transitional zone (El Obeid, Darfur)
- **South:** Wetter zone (Blue Nile, South Kordofan)

Each region has four CSV files:
- Average temperature (`T2M`)
- Minimum temperature (`T2M_MIN`)
- Maximum temperature (`T2M_MAX`)
- Precipitation (`PRECTOT`)

In total, there are 20 raw files.

---

## Cleaning and Preparation Strategy

Each dataset contains **monthly climate observations** for a given region and variable.  
Our cleaning process will:

1. **Load all CSV files** from the `Raw_datasets` folder.  
2. **Inspect** column names, date formats, and value ranges.  
3. **Clean** data by renaming inconsistent headers, parsing dates, and handling missing or invalid values.  
4. **Standardize** structure across all files (columns, units, and metadata).  
5. **Merge** datasets for further analysis by variable or by region.  
6. **Export** cleaned data to `cleaned_datasets` folder for use in Milestone 3 (Analysis).


### Setup and Imports

In [3]:
import pandas as pd
import numpy as np
import glob
import os

# Path to the folder containing the raw CSV files
data_path = "../1_datasets/Raw_datasets"  

# Get all CSV file paths from the folder
files = glob.glob(os.path.join(data_path, "*.csv"))

# Display how many files were found and their names
print(f"Found {len(files)} raw CSV files:")
for f in files:
    print("-", os.path.basename(f))


Found 20 raw CSV files:
- central_precip.csv
- central_t2m.csv
- central_t2mmax.csv
- central_t2mmin.csv
- east_precip.csv
- east_t2m.csv
- east_t2mmax.csv
- east_t2mmin.csv
- north_precip.csv
- north_t2m.csv
- north_t2mmax.csv
- north_t2mmin.csv
- south_precip.csv
- south_t2m.csv
- south_t2mmax.csv
- south_t2mmin.csv
- west_precip.csv
- west_t2m.csv
- west_t2mmax.csv
- west_t2mmin.csv


### Reading NASA POWER Files with Metadata

NASA POWER climate datasets include descriptive metadata at the top of each file before the actual data table starts.  
If loaded directly with `pandas.read_csv`, these metadata lines cause parsing errors because they don't follow standard CSV structure.

To handle this, we define a **custom loader function** that:
1. Reads the file as plain text.
2. Detects the first line that contains real table headers (e.g., "YEAR", "PARAMETER", "LAT", "LON").
3. Reads the file from that line onward into a clean `DataFrame`.

This ensures every file is parsed correctly, even when metadata length or format varies slightly.


In [4]:
import io

def read_nasa_power_csv(path, header_markers=("PARAMETER", "YEAR", "lat")):
    """
    Read NASA POWER CSV-like files that contain a metadata block before the real table.
    The function scans the file for a line containing one of header_markers, treats that
    as the header line, and loads the CSV from there.
    Returns a DataFrame.
    """
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()

    # find header line index: look for a line that contains commas and the keyword 'YEAR' or 'PARAMETER'
    header_idx = None
    for i, line in enumerate(lines):
        low = line.lower()
        if ("," in line) and any(h.lower() in low for h in header_markers):
            header_idx = i
            break

    if header_idx is None:
        # fallback: try to find the first line that looks like CSV (has many commas)
        for i, line in enumerate(lines):
            if line.count(",") >= 3:
                header_idx = i
                break

    if header_idx is None:
        raise ValueError(f"Could not detect header line in {path!r}")

    # join the rest of the file from header_idx and read with pandas
    data_str = "".join(lines[header_idx:])
    df = pd.read_csv(io.StringIO(data_str), sep=",", engine="python", on_bad_lines="skip")
    return df

# test on the sample file
sample_file = [f for f in files if "central_t2mmax" in f][0]
print("Trying to parse:", sample_file)
df_sample = read_nasa_power_csv(sample_file)
print("Parsed shape:", df_sample.shape)
display(df_sample.head(5))
print("Columns:", df_sample.columns.tolist())


Trying to parse: ../1_datasets/Raw_datasets\central_t2mmax.csv
Parsed shape: (1050, 17)


,PARAMETER,YEAR,LAT,LON,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANN
0,T2M_MAX,1990,14.0,31.250,35.87,35.31,38.36,44.11,43.93,43.28,40.14,40.44,41.39,41.22,39.57,38.55,44.11
1,T2M_MAX,1990,14.0,31.875,36.47,35.64,38.90,44.80,44.46,43.14,39.96,40.35,40.88,41.57,40.13,39.22,44.80
2,T2M_MAX,1990,14.0,32.500,36.38,35.32,38.69,43.91,44.14,42.40,38.28,39.42,40.83,41.55,39.71,38.75,44.14
3,T2M_MAX,1990,14.0,33.125,37.68,36.76,40.15,44.91,44.95,42.98,38.70,39.35,41.60,42.38,40.54,40.31,44.95
4,T2M_MAX,1990,14.0,33.750,37.08,36.27,39.51,43.45,43.81,41.65,37.12,37.13,39.22,41.05,39.93,39.91,43.81


Columns: ['PARAMETER', 'YEAR', 'LAT', 'LON', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC', 'ANN']


### Load Raw NASA POWER Datasets

In this step, we load all the CSV files from the `Raw_datasets` folder using our custom function.  
Each file contains temperature or rainfall data for different coordinates across Sudan.


In [5]:
# --- Load all NASA POWER CSV files from the Raw_datasets folder ---
import os

raw_folder = "../1_datasets/Raw_datasets" 

datasets = {}
for filename in os.listdir(raw_folder):
    if filename.endswith(".csv"):
        path = os.path.join(raw_folder, filename)
        try:
            df = read_nasa_power_csv(path)
            datasets[filename] = df
            print(f"Loaded {filename} | Shape: {df.shape}")
        except Exception as e:
            print(f"Could not read {filename}: {e}")

# --- Preview one dataset ---
if datasets:
    first_name = list(datasets.keys())[0]
    print(f"\n Preview of {first_name}:")
    display(datasets[first_name].head())
else:
    print("No datasets loaded. Check folder path or file types.")


Loaded central_precip.csv | Shape: (1050, 17)
Loaded central_t2m.csv | Shape: (1050, 17)
Loaded central_t2mmax.csv | Shape: (1050, 17)
Loaded central_t2mmin.csv | Shape: (1050, 17)
Loaded east_precip.csv | Shape: (980, 17)
Loaded east_t2m.csv | Shape: (980, 17)
Loaded east_t2mmax.csv | Shape: (980, 17)
Loaded east_t2mmin.csv | Shape: (980, 17)
Loaded north_precip.csv | Shape: (1715, 17)
Loaded north_t2m.csv | Shape: (1715, 17)
Loaded north_t2mmax.csv | Shape: (1715, 17)
Loaded north_t2mmin.csv | Shape: (1715, 17)
Loaded south_precip.csv | Shape: (1225, 17)
Loaded south_t2m.csv | Shape: (1225, 17)
Loaded south_t2mmax.csv | Shape: (1225, 17)
Loaded south_t2mmin.csv | Shape: (1225, 17)
Loaded west_precip.csv | Shape: (1715, 17)
Loaded west_t2m.csv | Shape: (1715, 17)
Loaded west_t2mmax.csv | Shape: (1715, 17)
Loaded west_t2mmin.csv | Shape: (1715, 17)

 Preview of central_precip.csv:


,PARAMETER,YEAR,LAT,LON,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANN
0,PRECTOTCORR,1990,14.0,31.250,0.0,0.0,0.0,0.0,0.00,0.05,3.18,0.85,0.49,0.18,0.0,0.0,0.40
1,PRECTOTCORR,1990,14.0,31.875,0.0,0.0,0.0,0.0,0.00,0.05,3.26,0.93,0.54,0.24,0.0,0.0,0.42
2,PRECTOTCORR,1990,14.0,32.500,0.0,0.0,0.0,0.0,0.00,0.04,2.84,1.16,0.88,0.76,0.0,0.0,0.48
3,PRECTOTCORR,1990,14.0,33.125,0.0,0.0,0.0,0.0,0.01,0.05,2.69,1.31,1.08,1.12,0.0,0.0,0.53
4,PRECTOTCORR,1990,14.0,33.750,0.0,0.0,0.0,0.0,0.01,0.11,3.24,1.61,1.46,1.41,0.0,0.0,0.66


### Inspect and Profile Each Dataset

Now that all datasets are loaded, we inspect their structure and quality.  
We will:
- Check column names and data types.  
- Look for missing or invalid values.  
- Verify that all datasets share the same schema (PARAMETER, YEAR, LAT, LON, JAN–DEC, ANN).  

This step helps confirm data consistency before cleaning or merging.


In [6]:
for name, df in datasets.items():
    print(f"{name}")
    print("-" * 60)
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print(df.describe(include='all').T[['count', 'mean', 'min', 'max']].head(5))
    print("\nMissing values per column:")
    print(df.isnull().sum())
    print("=" * 60, "\n")


central_precip.csv
------------------------------------------------------------
Shape: (1050, 17)
Columns: ['PARAMETER', 'YEAR', 'LAT', 'LON', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC', 'ANN']
            count      mean     min     max
PARAMETER    1050       NaN     NaN     NaN
YEAR       1050.0    2007.0  1990.0  2024.0
LAT        1050.0     15.25    14.0    16.5
LON        1050.0      32.5   31.25   33.75
JAN        1050.0  0.000457     0.0    0.06

Missing values per column:
PARAMETER    0
YEAR         0
LAT          0
LON          0
JAN          0
FEB          0
MAR          0
APR          0
MAY          0
JUN          0
JUL          0
AUG          0
SEP          0
OCT          0
NOV          0
DEC          0
ANN          0
dtype: int64

central_t2m.csv
------------------------------------------------------------
Shape: (1050, 17)
Columns: ['PARAMETER', 'YEAR', 'LAT', 'LON', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'O

### Merge Regional Datasets into National-Level Files

Each region of Sudan (Central, North, East, South, and West) has its own temperature and precipitation datasets.  
In this step, we merge them into two complete national-level datasets:
- **Temperature dataset** — combines all T2M (Temperature at 2 Meters) files.
- **Precipitation dataset** — combines all PRECTOT (Rainfall) files.

Each record will include a new column `REGION` indicating the source region.  
The merged datasets will be saved in the `Cleaned_datasets` folder for further analysis.


In [7]:

raw_folder = "../1_datasets/Raw_datasets"
clean_folder = "../1_datasets/Cleaned_datasets"

os.makedirs(clean_folder, exist_ok=True)

merged_temp = []
merged_precip = []

for filename in os.listdir(raw_folder):
    if not filename.endswith(".csv"):
        continue
    
    path = os.path.join(raw_folder, filename)
    df = read_nasa_power_csv(path)

    # Extract region name from filename (e.g., "central_t2m.csv" -> "Central")
    region = filename.split("_")[0].capitalize()
    df["REGION"] = region

    # Separate temperature and precipitation
    if "t2m" in filename.lower():
        merged_temp.append(df)
    elif "precip" in filename.lower():
        merged_precip.append(df)

# Concatenate all temperature and precipitation data
df_temp = pd.concat(merged_temp, ignore_index=True)
df_precip = pd.concat(merged_precip, ignore_index=True)

# Save merged files
temp_path = os.path.join(clean_folder, "merged_temperature.csv")
precip_path = os.path.join(clean_folder, "merged_precipitation.csv")

df_temp.to_csv(temp_path, index=False)
df_precip.to_csv(precip_path, index=False)

print(f"Saved merged temperature data: {temp_path} ({df_temp.shape[0]} rows)")
print(f"Saved merged precipitation data: {precip_path} ({df_precip.shape[0]} rows)")


Saved merged temperature data: ../1_datasets/Cleaned_datasets\merged_temperature.csv (20055 rows)
Saved merged precipitation data: ../1_datasets/Cleaned_datasets\merged_precipitation.csv (6685 rows)
